Here's what this notebook does, step-by-step:

1. Install the `montreal-force-aligner` (MFA) library using conda.
2. Download the Viet Bible Vox dataset from Huggingface.
3. Generate a normalized transcript file for each .mp3 file in the dataset.
4. Train an MFA model, then align speech and phonemes (creating a timestamp for each phoneme).
5. Generate a TensorFlow dataset (tfrecord files) that is ready for training TTS models.

In [1]:
# %pip install -Uqq tensorflow numpy torch tqdm librosa numba regex

In [2]:
# ./data is our working directory
# !mkdir -p data
%cd data

/home/billgates/lightspeed/data


In [3]:
#### INSTALL MFA  ####
# !wget https://repo.anaconda.com/miniconda/Miniconda3-py311_23.5.2-0-Linux-x86_64.sh -qO ./miniconda.sh
# # !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -qO $PWD/miniconda.sh # for Apple M1
# !bash ./miniconda.sh -b -p ./miniconda
# !source ./miniconda/bin/activate && conda create -n aligner -c conda-forge montreal-forced-aligner=2.2.15 -y --quiet
# if done install
# !source ./miniconda/bin/activate aligner

In [4]:
!pwd

/home/billgates/lightspeed/data


In [6]:
# !git lfs install # LFS is needed to download the data
# !git clone https://huggingface.co/datasets/ntt123/VietBibleVox
# !cd audio_1; unzip -qq audio_1.zip
!cd audio_1_experiment/audio_1_ver2; ls *.wav | wc -l
!cd audio_1_experiment/audio_1_ver2; ls *.txt | wc -l

2120
2120


In [17]:
import regex
import unicodedata
from pathlib import Path

In [18]:
vietnamese_characters = [
    'a', 'à', 'á', 'ả', 'ã', 'ạ',
    'ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ',
    'â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ',
    'e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ',
    'ê', 'ề', 'ế', 'ể', 'ễ', 'ệ',
    'i', 'ì', 'í', 'ỉ', 'ĩ', 'ị',
    'o', 'ò', 'ó', 'ỏ', 'õ', 'ọ',
    'ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ',
    'ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ',
    'u', 'ù', 'ú', 'ủ', 'ũ', 'ụ',
    'ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự',
    'y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ',
    'b', 'c', 'd', 'đ', 'g', 'h', 
    'k', 'l', 'm', 'n', 'p', 'q', 
    'r', 's', 't', 'v', 'x'
]
alphabet = "".join(vietnamese_characters)
space_re = regex.compile(r"\s+")
number_re = regex.compile("([0-9]+)")
digits = ["không", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
num_re = regex.compile(r"([0-9.,]*[0-9])")
keep_text_and_num_re = regex.compile(rf'[^\s{alphabet}.,0-9]')
keep_text_re = regex.compile(rf'[^\s{alphabet}]')

In [19]:
def read_number(num: str) -> str:
    if len(num) == 1:
        return digits[int(num)]
    elif len(num) == 2 and num.isdigit():
        n = int(num)
        end = digits[n % 10]
        if n == 10:
            return "mười"
        if n % 10 == 5:
            end = "lăm"
        if n % 10 == 0:
            return digits[n // 10] + " mươi"
        elif n < 20:
            return "mười " + end
        else:
            if n % 10 == 1:
                end = "mốt"
            return digits[n // 10] + " mươi " + end
    elif len(num) == 3 and num.isdigit():
        n = int(num)
        if n % 100 == 0:
            return digits[n // 100] + " trăm"
        elif num[1] == "0":
            return digits[n // 100] + " trăm lẻ " + digits[n % 100]
        else:
            return digits[n // 100] + " trăm " + read_number(num[1:])
    elif len(num) >= 4 and len(num) <= 6 and num.isdigit():
        n = int(num)
        n1 = n // 1000
        return read_number(str(n1)) + " ngàn " + read_number(num[-3:])
    elif "," in num:
        n1, n2 = num.split(",")
        return read_number(n1) + " phẩy " + read_number(n2)
    elif "." in num:
        parts = num.split(".")
        if len(parts) == 2:
            if parts[1] == "000":
                return read_number(parts[0]) + " ngàn"
            elif parts[1].startswith("00"):
                end = digits[int(parts[1][2:])]
                return read_number(parts[0]) + " ngàn lẻ " + end
            else:
                return read_number(parts[0]) + " ngàn " + read_number(parts[1])
        elif len(parts) == 3:
            return (
                read_number(parts[0])
                + " triệu "
                + read_number(parts[1])
                + " ngàn "
                + read_number(parts[2])
            )
    return num

In [20]:
def normalize_text(x):
    x = unicodedata.normalize('NFKC', x)
    x = x.lower()
    x = num_re.sub(r" \1 ", x)
    x = keep_text_and_num_re.sub(" ", x)
    words = x.split()
    words = [ read_number(w) if num_re.fullmatch(w) else w for w in words ]
    x = " ".join(words)
    x = keep_text_re.sub(" ", x)
    x = space_re.sub(" ", x)
    x = x.strip()
    return x

In [21]:
all_text = []
for fp in sorted(Path("audio_1_experiment/audio_1_ver2").glob("*.txt")):
    with open(fp, "r", encoding="utf-8") as f:
        text = f.read()
        # print(text)
        text = normalize_text(text)
        all_text.append(text)
    # override the text file
    with open(fp, "w", encoding="utf-8") as f:
        f.write(text)
all_words = sorted(set((" ".join(all_text)).split()))

In [22]:
with open("lexicon.txt", "w") as f:
    for w in all_words:
        w = w.strip()
        p = list(w)
        p = " ".join(p)
        f.write(f"{w}\t{p}\n")

In [23]:
%cd data

[Errno 2] No such file or directory: 'data'
/home/billgates/lightspeed/data


In [24]:
!source miniconda/bin/activate aligner; \
mfa train \
    --num_jobs `nproc` \
    --use_mp \
    --clean \
    --overwrite \
    --no_textgrid_cleanup \
    --single_speaker \
    --output_format json \
    --output_directory audio_1_experiment/audio_1_ver2 \
    audio_1_experiment/audio_1_ver2 ./lexicon.txt audio_1_experiment/audio_1_ver2_mfa

Please be aware that you are running an alpha version of MFA. If you would like to install a more stable version, please visit https://montreal-forced-aligner.readthedocs.io/en/latest/installation.html#installing-older-versions-of-mfa
 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1,711/100  [ 0:00:00 < 0:00:00 , 9,116 it/s ]
 INFO     Found 1 speaker across 2120 files, average number of utterances per   
          speaker: 2120.0                                                       
 INFO     Initializing multiprocessing jobs...                                  
 INFO     Normalizing text...                                                   
  38% ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 811/2,120  [ 0:00:02 < -:--:-- , ? it/s ]
 INFO     Generating MFCCs...                                                   
  96% ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 2

In [25]:
#### PREPARE TF DATASET ####
# Note: We are using a sampling rate of 16k,
# even though the original data has a sampling rate of 48k.

In [26]:
import json
from pathlib import Path
import numpy as np
import torch
import json
import librosa
import tensorflow as tf
from tqdm.auto import tqdm
import random

/home/billgates/miniconda3/envs/TTS_experiment_3/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-25 10:55:12.112627: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-25 10:55:12.790540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-25 10:55:12.790604: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library '

In [27]:
import json
from pathlib import Path
import numpy as np
import torch
import json
import librosa
import tensorflow as tf
from tqdm.auto import tqdm
import random

In [40]:
mel_basis = {}
hann_window = {}

def spectrogram_torch(y, n_fft, sampling_rate, hop_size, win_size, center=False):
    global hann_window
    # if torch.min(y) < -1.0:
    #     print("min value is ", torch.min(y))
    # if torch.max(y) > 1.0:
    #     print("max value is ", torch.max(y))

    dtype_device = str(y.dtype) + "_" + str(y.device)
    wnsize_dtype_device = str(win_size) + "_" + dtype_device
    if wnsize_dtype_device not in hann_window:
        hann_window[wnsize_dtype_device] = torch.hann_window(win_size).to(
            dtype=y.dtype, device=y.device
        )

    y = torch.nn.functional.pad(
        y.unsqueeze(1),
        (int((n_fft - hop_size) / 2), int((n_fft - hop_size) / 2)),
        mode="reflect",
    )
    y = y.squeeze(1)

    spec = torch.stft(
        y,
        n_fft,
        hop_length=hop_size,
        win_length=win_size,
        window=hann_window[wnsize_dtype_device],
        center=center,
        pad_mode="reflect",
        normalized=False,
        onesided=True,
        return_complex=True
    )
    spec = torch.view_as_real(spec)
    spec = torch.sqrt(spec.pow(2).sum(-1) + 1e-6)
    spec = spec.squeeze(0)
    return torch.swapaxes(spec, 0, 1)


def tensor_to_bytes(t):
    t = tf.constant(t)
    t = tf.io.serialize_tensor(t)
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[t.numpy()]))


def write_tfdata(data, out_file):
    with tf.io.TFRecordWriter(out_file) as file_writer:
        for wav_file, seq, _ in data:
            phone_seq = []
            for phone, duration in seq:
                phone_idx = phone_set.index(phone)
                phone_seq.append((phone_idx, duration))
            phone_seq = np.array(phone_seq, dtype=np.float32)

            # load wav
            wav, sr = librosa.load(wav_file, sr=config["data"]["sampling_rate"], dtype=np.float32)
            wav = torch.from_numpy(wav)
            # compute spec
            spec = spectrogram_torch(
                wav[None],
                n_fft=config["data"]["filter_length"],
                sampling_rate=config["data"]["sampling_rate"],
                hop_size=config["data"]["hop_length"],
                win_size=config["data"]["win_length"],
                center=False
            )
            features = {
                "phone_idx": tensor_to_bytes(phone_seq[:, 0].astype(np.int32)),
                "phone_duration": tensor_to_bytes(phone_seq[:, 1]),
                "wav": tensor_to_bytes(wav.half().numpy()),
                "spec": tensor_to_bytes(spec.half().numpy())
            }
            example = tf.train.Example(features=tf.train.Features(feature=features))
            file_writer.write(example.SerializeToString())

def write_split(split, data, num_chunks):
    data = np.array(data, dtype=object)
    chunks = list(np.array_split(data, num_chunks))
    for i, chunk in enumerate(tqdm(chunks)):
        write_tfdata(chunk, f"tfdata_audio_1/{split}/part_{i:03d}.tfrecords")

In [41]:
!mkdir -p tfdata_audio_1/{train,test}

In [42]:
%cd data

[Errno 2] No such file or directory: 'data'
/home/billgates/lightspeed/data


In [43]:
with open("../config.json", "rb") as f:
    config = json.load(f)
device = "cuda" if torch.cuda.is_available() else "cpu"
data_dir = Path("audio_1_experiment/audio_1_ver2/")
json_files = sorted(data_dir.glob("*.json"))
dataset = []
phone_set = []

for file_path in json_files:
    with open(file_path, "rb") as f:
        data = json.load(f)
    seq = []
    word_index = 0
    words =  data["tiers"]["words"]["entries"]
    for start, end, phone in data["tiers"]["phones"]["entries"]:
        if start > words[word_index][1] - 1e-5:
            seq.append( ("<SEP>", 0) )
            word_index += 1
        duration = end * 1000 - start * 1000 # ms
        phone_set.append(phone)
        seq.append( (phone, duration) )
    # print("seq", seq)
    wav_file = file_path.with_suffix(".wav")
    dataset.append((wav_file, seq, data["end"]))

phone_set = ["<SEP>"] + sorted(set(phone_set))
assert len(phone_set) <= 256
with open("phone_set.json", "w", encoding="utf-8") as f:
    json.dump(phone_set, f)

assert phone_set.index("<SEP>") == 0

In [44]:
random.Random(42).shuffle(dataset)
L = len(dataset) - 256
train_data = dataset[:L]
test_data = dataset[L:]
print("Train data size:", len(train_data))
print("Test data size:", len(test_data))

Train data size: 1860
Test data size: 256


In [45]:
%pwd

'/home/billgates/lightspeed/data'

In [46]:
train_data[0]

(PosixPath('audio_1_experiment/audio_1_ver2/audio_1-5827.wav'),
 [('c', 10.0),
  ('ả', 10.0),
  ('m', 10.0),
  ('<SEP>', 0),
  ('t', 20.0),
  ('h', 10.0),
  ('ấ', 10.0),
  ('y', 50.0),
  ('<SEP>', 0),
  ('sil', 80.0),
  ('<SEP>', 0),
  ('g', 10.0),
  ('h', 10.0),
  ('e', 90.0),
  ('n', 120.0),
  ('<SEP>', 0),
  ('t', 60.0),
  ('ị', 180.0),
  ('<SEP>', 0),
  ('v', 60.0),
  ('ớ', 50.0),
  ('i', 60.0),
  ('<SEP>', 0),
  ('h', 130.0),
  ('ọ', 320.0),
  ('<SEP>', 0),
  ('sil', 100.0),
  ('<SEP>', 0),
  ('h', 100.0),
  ('ỏ', 40.0),
  ('i', 30.0),
  ('<SEP>', 0),
  ('sil', 30.0),
  ('<SEP>', 0),
  ('đ', 50.0),
  ('ộ', 140.0),
  ('c', 20.0),
  ('<SEP>', 0),
  ('l', 90.0),
  ('ậ', 60.0),
  ('p', 20.0),
  ('<SEP>', 0),
  ('sil', 210.0),
  ('<SEP>', 0),
  ('t', 60.0),
  ('ự', 180.0),
  ('<SEP>', 0),
  ('t', 40.0),
  ('i', 70.0),
  ('n', 130.0),
  ('<SEP>', 0),
  ('sil', 30.0),
  ('<SEP>', 0),
  ('v', 70.0),
  ('à', 10.0),
  ('<SEP>', 0),
  ('sil', 60.0),
  ('<SEP>', 0),
  ('đ', 40.0),
  ('ư', 70.

In [47]:
write_split("test", test_data, 1)

  0%|          | 0/1 [00:00<?, ?it/s]2024-01-25 10:56:34.714317: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-25 10:56:34.714711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2024-01-25 10:56:34.714785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2024-01-25 10:56:34.714851: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2024-01-25 

In [48]:
write_split("train", train_data, 256)

100%|██████████| 256/256 [00:08<00:00, 30.09it/s]
